<a href="https://colab.research.google.com/github/LxYuan0420/nlp/blob/main/notebooks/Evaluator_from_Huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers datasets evaluate

In [2]:
import evaluate
from evaluate import evaluator
from datasets import load_dataset

task_evaluator = evaluator("text-classification")
data = load_dataset("banking77")

/root/.cache/huggingface/modules/datasets_modules/datasets/banking77/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4/banking77.py:59: FutureWarning: Dataset 'banking77' is deprecated and will be deleted. Use 'PolyAI/banking77' instead.
  warnings.warn(


Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

Dataset banking77 downloaded and prepared to /root/.cache/huggingface/datasets/banking77/default/1.1.0/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10003
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3080
    })
})

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("lxyuan/banking-intent-distilbert-classifier")

model = AutoModelForSequenceClassification.from_pretrained("lxyuan/banking-intent-distilbert-classifier")

In [41]:
# currently this feature development is on-hold so we will use sklearn to do the evaluation
# https://discuss.huggingface.co/t/combining-metrics-for-multiclass-predictions-evaluations/21792/11
results = task_evaluator.compute(
    model_or_pipeline=model,
    data=data["test"],
    metric=evaluate.combine([
        "accuracy",
#        evaluate.load("precision", average="macro"),
#        evaluate.load("recall", average="macro"),
#        evaluate.load("f1", average="macro") 
    ]),
    tokenizer=tokenizer,
    label_mapping=model.config.label2id,
    strategy="simple",
)

In [43]:
results

{'accuracy': 0.9243506493506494,
 'total_time_in_seconds': 30.070161925999855,
 'samples_per_second': 102.42711720607363,
 'latency_in_seconds': 0.009763039586363589}

In [21]:
x_test = [text for text in data["test"]["text"]]
y_test = [label for label in data["test"]["label"]]

In [25]:
from transformers import pipeline
text_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
y_pred = text_classifier(x_test)

In [30]:
y_pred = [model.config.label2id[pred['label']] for pred in y_pred]
y_pred[:5]

[41, 11, 11, 12, 11]

In [42]:
from sklearn.metrics import classification_report

label_names = [label for id, label in model.config.id2label.items()]
report = classification_report(y_test, y_pred, target_names=label_names, digits=4)
print("Classification Report:\n", report)

Classification Report:
                                                   precision    recall  f1-score   support

                                activate_my_card     1.0000    0.9750    0.9873        40
                                       age_limit     0.9756    1.0000    0.9877        40
                         apple_pay_or_google_pay     1.0000    1.0000    1.0000        40
                                     atm_support     0.9750    0.9750    0.9750        40
                                automatic_top_up     1.0000    0.9000    0.9474        40
         balance_not_updated_after_bank_transfer     0.8205    0.8000    0.8101        40
balance_not_updated_after_cheque_or_cash_deposit     1.0000    0.9750    0.9873        40
                         beneficiary_not_allowed     0.9250    0.9250    0.9250        40
                                 cancel_transfer     1.0000    0.9750    0.9873        40
                            card_about_to_expire     0.9756    1.0000    0.